In [4]:
import os


In [5]:
%pwd

'g:\\Text-Summarization-Project\\research'

In [6]:
os.chdir('../')

In [7]:
%pwd

'g:\\Text-Summarization-Project'

In [14]:
from dataclasses import dataclass
from pathlib import Path

class DataIngestionConfig:
    def __init__(self, root_dir: str, source_URL: str, local_data_file: str, unzip_dir: str):
        self.root_dir = root_dir
        self.source_URL = source_URL
        self.local_data_file = local_data_file
        self.unzip_dir = unzip_dir


In [15]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [16]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [17]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Zip file extracted to: {unzip_path}")

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-05 15:21:26,334: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-05 15:21:26,337: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-05 15:21:26,338: INFO: common: created directory at: artifacts]
[2024-08-05 15:21:26,339: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-05 15:21:43,583: INFO: 2408082172: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9262:2F8C56:364C2B:42BFBA:66B0A7A9
Accept-Ranges: bytes
Date: Mon, 05 Aug 2024 10:21:29 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg120111-QPG
X-Cache: 